In [ ]:
# 定义一个函数来计算两点之间的距离
def calculate_distance(point1, point2,scale=10):
    return ee.Number(point1.get('x')).subtract(point2.get('x')).multiply(ee.Number(scale)).pow(2) \
        .add(ee.Number(point1.get('y')).subtract(point2.get('y')).multiply(ee.Number(scale)).pow(2)).sqrt()
        

# 定义一个函数来计算极值点与其前十位数的差值和角度
def calculate_Layover(index,pointList,Candidate_Distor_Points=10,scale=10):
    
    # 获取极值点
    extreme_point = ee.Dictionary(pointList.get(index))

    # 获取极值点前十位数的索引范围及前十位数的点
    start_index_left = ee.Number(index).subtract(Candidate_Distor_Points).max(0)  # 确保索引不小于 0
    end_index_left = index
    previous_points_left = pointList.slice(start_index_left, end_index_left)
    
    # ----------------------------------------Left layover----------------------------------------
    # 计算 elevation 差值、距离差值\计算角度和 Distortion\判断是否存在几何畸变
    differences_Left = previous_points_left.map(lambda point:
        ee.Dictionary({
            'elevation_difference': ee.Number(extreme_point.get('elevation')).subtract(ee.Number(ee.Dictionary(point).get('elevation'))),
            'distance': calculate_distance(extreme_point, ee.Dictionary(point),scale=scale),
            'point_coordinates':extreme_point.get('point_coordinates'),
            'Distortion Type':'Leftlayover'}))

    differences_and_angle_Left = differences_Left.map(lambda item:
        ee.Dictionary(item).set('arcAngle', ee.Algorithms.If(
            ee.Number(ee.Dictionary(item).get('elevation_difference')).gt(0),  # 检查高差是否为正
            ee.Number(ee.Dictionary(item).get('distance')).atan2(ee.Dictionary(item).get('elevation_difference')).multiply(180).divide(ee.Number(math.pi)),  # 计算角度（度）
            -1  # 如果高差为负，不计算角度
        )))

    differences_and_angle_Left = differences_and_angle_Left.map(lambda item:
            ee.Dictionary(item).set(
                    'Distortion', ee.Algorithms.If(
                    ee.Number(ee.Dictionary(item).get('arcAngle')).gt(extreme_point.get('angle')),
                    1,0)))
    
    features_Left = differences_and_angle_Left.map(lambda item:
        ee.Feature(ee.Geometry.Point(ee.List(ee.Dictionary(item).get('point_coordinates'))), item))
    
    # ----------------------------------------Right layover----------------------------------------

    # 筛选出存在 Leftlayover 几何畸变的特征，计算RightLayover几何畸变
    distorted_features_Left = ee.FeatureCollection(features_Left).filter(ee.Filter.eq('Distortion', 1))

    # 按距离降序排序并获取距离最远的特征
    max_distance_feature = distorted_features_Left.sort('distance', False).first()

    # 获取最远特征的高程
    max_distance_elevation = ee.Number(max_distance_feature.get('elevation_difference'))

    # 创建一个新的字典并替换 'elevation'
    new_extreme_point = extreme_point.set('elevation', ee.Number(extreme_point.get('elevation')).subtract(max_distance_elevation))

    start_index_Right = ee.Number(index).add(Candidate_Distor_Points).min(pointList.size().subtract(1))  # 确保索引不大于 pointList 的最大索引
    end_index_Right = index
    afters_points_Right = pointList.slice(ee.Number(end_index_Right).add(1),start_index_Right.add(1)).reverse()

    differences_Right = afters_points_Right.map(lambda point:
        ee.Dictionary({
            'elevation_difference': ee.Number(ee.Dictionary(point).get('elevation')).subtract(ee.Number(new_extreme_point.get('elevation'))),
            'distance': calculate_distance(new_extreme_point, ee.Dictionary(point),scale=scale),
            'point_coordinates':new_extreme_point.get('point_coordinates'),
            'Distortion Type':'Rightlayover'}))

    differences_and_angle_Right = differences_Right.map(lambda item:
        ee.Dictionary(item).set('arcAngle', ee.Algorithms.If(
            ee.Number(ee.Dictionary(item).get('elevation_difference')).gt(0),  # 检查高差是否为正
            ee.Number(ee.Dictionary(item).get('distance')).atan2(ee.Dictionary(item).get('elevation_difference')).multiply(180).divide(ee.Number(math.pi)),  # 计算角度（度）
            -1  # 如果高差为负，不计算角度
        )))

    differences_and_angle_Right = differences_and_angle_Right.map(lambda item:
            ee.Dictionary(item).set(
                'Distortion', ee.Algorithms.If(
                    ee.Number(ee.Dictionary(item).get('arcAngle')).gt(extreme_point.get('angle')),
                    1,0)))
    
    features_Right = differences_and_angle_Right.map(lambda item:
        ee.Feature(ee.Geometry.Point(ee.List(ee.Dictionary(item).get('point_coordinates'))), item))
    
    # 筛选出存在几何畸变的特征
    distorted_features_Right = ee.FeatureCollection(features_Right).filter(ee.Filter.eq('Distortion', 1))

    # 判断是否存在几何畸变的特征,将顶点追加进去
    has_distortion = ee.Algorithms.If(
        distorted_features_Right.size().gt(0),
        features_Right.add(ee.Feature(
            ee.Geometry.Point(ee.List(extreme_point.get('point_coordinates'))),
            {'Distortion': 1}
        )),
        features_Right
    )

    # 将结果转换为列表
    features_Right = ee.List(has_distortion)
    
    return [features_Left,features_Right]

Candidate_Distor_Points = 10
scale_=10
# 使用 unionFilter_indices 映射函数
features = unionFilter_indices.map(lambda index: calculate_Layover(
                                            ee.Number(index), pointList=test, Candidate_Distor_Points=Candidate_Distor_Points, 
                                            scale=scale_)).flatten()
Leftlayover = ee.FeatureCollection(features).filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Leftlayover')))
Rightlayover = ee.FeatureCollection(features).filter(ee.Filter.And(
                    ee.Filter.eq('Distortion', 1),
                    ee.Filter.eq('Distortion Type', 'Rightlayover')))

# 构建所有点？
# features_collection = ee.FeatureCollection(features).filter(ee.Filter.eq('Distortion', 1))

In [ ]:
# 根据union_indices 在 Points_WithH_Angle上确定极值点
test = ee.List(Points_WithH_Angle.get(0))
extracted_elements = unionFilter_indices.map(lambda index: ee.Dictionary(test.get(index)).set('index', index))
extracted_elements